# AFSK Demodulator
## Step 6: HDLC Decoder

-----

This notebook will outline the steps necessary to move the HDLC Decoder to FPGA.  This will be the final step in our project.  Like with our last one, we will be borrowing code from the Mobilinkd TNC3 firmware.  However, we do need to write a CRC calculator.  In the TNC3, we use the hardware IP to calculate it.  And in the Python implementation, we use a lookup table, which would work but would be inefficient on an FPGA.

This code is part of the [AFSK Demodulator on Pynq](afsk-demodulator-fpga.ipynb) project.

The purpose of this code is to continue our migration of the Python demodulator code to FPGA.  We will be streaming audio data into the FPGA and streaming processed data out from the FPGA.

This is the third step of moving a demodulator processing step into the FPGA. At this point demodulation is being done in FPGA.  We are left with clock recovery and HDLC framing.  Here we address clock recovery.

At this point we must diverge from the design pattern we have been following.  No longer are we simply streaming data in and out.  The PLL had to indicate *lock* status, it has to output a *sample* indicator.  And it may need to output information about *jitter* for diagnostic purposes.

The Digital PLL in Python provides all of these interfaces.  However, we can change the interface.  We only need to provide two outputs from the PLL: a stream of sampled bits, and a lock indicator.  Audio data will be clocked in via a stream interface.  This will be demodulated to a bitstream and processed by the digital PLL.  The demodulator will clock out 3 bits for each audio sample: the demodulated bit, the a lock flag, and a sample indicator.  The sample indicator will never go high if the lock flag is low.

Recall from the Python implementation of the PLL that we need an IIR filter and a hysteresis module.  We will build and test these independently.  The PLL also made use of floating point math in the PLL, IIR filter and hysteresis code.  We will change that to fixed point.

## Prerequisites

At this point you are expected to have:

 * A configured PYNQ environment.
 * Vivado installed on your computer and configured for your board.
 * Experience working through the tutorials at https://pynq.readthedocs.io/.
 * Familiarized yourself with the AFSK demodulator implementation in Python.
 * Completed the first four steps of the tutorial to familiarize yourself with the process of creating a streaming interface.

## Outline

We are going to modify the FPGA IP we created in the third tutorial to add the low-pass filter for the correlator output we are now generating, and turn that back into a bitstream.

We will perform the following steps in this section:

 1. Create a C++ file that accepts a block of 16-bit data, performs the FIR, correlator and low-pass filter operations,  and sends the resulting bitstream back.
 1. Create a C++ test case for the above file.
 1. Generate an IP package from the code that can be used in Vivado.
 1. Create a Zynq project in Vivado that uses the IP.
 1. Export the bitstream for our project from Vivado.
 1. Use Python running on the PS to load the bitstream to the PL, and verify that it works.
 1. Integrate the FPGA module with the existing demodulator code, replacing the existing Python code.

First we are going to generate the FIR filter coefficients.  Then we are going to generate some sample data for our test bench. 

## Filter Coefficients

We continue to generate the filter coefficents, because we still need to test against the Python implementation.  But we no longer need to print them out.  Our work with filters is complete.  We now focus on the digitl PLL.

In [9]:
import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read

audio_file = read('../base/TNC_Test_Ver-1.102-26400-1sec.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]

bpf_coeffs = np.array(firwin(141, [1100.0/(sample_rate/2), 2300.0/(sample_rate/2)], width = None,
        pass_zero = False, scale = True, window='hann') * 32768, dtype=int)

lpf_coeffs = np.array(firwin(101, [760.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

pll_lpf40_coeffs = np.array(firwin(5, [40.0/600.0], width = None,
        pass_zero = True, scale = True, window='hann'), dtype=float)
print("lpf40 =", ",".join([str(x) for x in pll_lpf40_coeffs]))

pll_lpf64_coeffs = np.array(firwin(5, [64.0/600.0], width = None,
        pass_zero = True, scale = True, window='hann'), dtype=float)
print("lpf64 =", ",".join([str(x) for x in pll_lpf64_coeffs]))

lpf40 = 0.0,0.24908481189362192,0.5018303762127563,0.24908481189362192,0.0
lpf64 = 0.0,0.24765178738120416,0.5046964252375916,0.24765178738120416,0.0


## Test Bench Data

We will now generate the input and output data for our test bench.  We will again use our working Python model to generate data as a baseline.  We need to generate PLL output data.  This is going to be a bit different than the data currently provided because we are changing the interface slightly.  We need to generate an array containing three numbers (bits) from the PLL: input, locked, sample.

In [15]:
import sys
sys.path.append('../base')
from DigitalPLL import DigitalPLL
from HDLC import HDLC

pll = DigitalPLL(sample_rate, 1200.0)

class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768, [], [])
    def __call__(self, data):
        result, self.zl = lfilter(self.coeffs, 32768, data, -1, self.zl)
        return result

bpf = fir_filter(bpf_coeffs)
lpf = fir_filter(lpf_coeffs)

delay = 12

f = bpf(audio_data[:26400])
c = np.array([int(x >= 0) for x in f])
# Delay the data
d = np.append(np.zeros(delay, dtype=int), np.array(c[:0-delay], dtype=int))
# XOR the digitized data with the delayed version
x = np.logical_xor(c, d)
l = lpf(x * 2 - 1)
comp = np.array([int(x >= 0) for x in l])

locked = np.zeros(len(comp), dtype=int)
sample = np.zeros(len(comp), dtype=int)

for i in range(len(comp)):
    sample[i] = pll(comp[i])
    locked[i] = pll.locked()

from HDLC import HDLC

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result


nrzi = NRZI()
hdlc = HDLC()

decoded = None

print("int16_t audio[] = {\n    ", end='')
count = 0
for x in audio_data[:26400]:
    print(hex(x), end=', ')
    count += 1
    if count == 16:
        print('\n    ', end='')
        count = 0
print("\n};")


for b,s,l in zip(comp, sample, locked):
    if s:
        packet = hdlc(nrzi(b), l)
        if packet is not None:
            print("uint8_t packet[] = {\n    ", end='')
            count = 0
            for x in packet[1]:
                print('0x%02x, ' % ord(x), end='')
                count += 1
                if count == 16:
                    print('\n    ', end='')
                    count = 0
            print("\n};")
            print(len(packet[1]))
            decoded = packet[1]
            print("0x%04x" % packet[0])

/usr/lib64/python3.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/usr/lib64/python3.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/usr/lib64/python3.7/site-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(s

int16_t audio[] = {
    0x2cf, 0x2ec, 0x1d4, 0x1e7, 0x215, 0x370, 0x4a3, 0x6b5, 0x84c, 0x8d6, 0x971, 0x943, 0x83a, 0x702, 0x4fb, 0x2b2, 
    0x3, -0x2d1, -0x566, -0x73f, -0x8b3, -0x94a, -0x94f, -0x8c3, -0x7a1, -0x5e6, -0x3be, -0x123, 0xd6, 0x1f1, 0x341, 0x38d, 
    0x332, 0x26c, 0x122, -0xcf, -0x313, -0x574, -0x7e3, -0x982, -0xac4, -0xb62, -0xb55, -0xaca, -0x978, -0x7a2, -0x55b, -0x29b, 
    -0x42, 0x10e, 0x27e, 0x2fa, 0x2fa, 0x2aa, 0x1ea, 0xeb, 0x64, 0xa1, 0x118, 0x247, 0x391, 0x56f, 0x628, 0x662, 
    0x695, 0x576, 0x445, 0x292, 0xff, 0x5e, 0x2, 0x69, 0x15d, 0x2f9, 0x508, 0x76a, 0x8ff, 0xa04, 0xae9, 0xab8, 
    0xa34, 0x8d8, 0x73b, 0x500, 0x24a, -0x8f, -0x33e, -0x538, -0x703, -0x7c9, -0x7f6, -0x77d, -0x656, -0x4b9, -0x286, 0x1c, 
    0x256, 0x3a1, 0x4f1, 0x566, 0x532, 0x4a6, 0x34b, 0x183, -0x9d, -0x308, -0x58c, -0x74a, -0x8b3, -0x94b, -0x92a, -0x891, 
    -0x74c, -0x581, -0x31c, -0x6f, 0x22d, 0x3b5, 0x564, 0x664, 0x644, 0x60e, 0x51e, 0x3b2, 0x1c1, -0x71, -0x2e8, -0x4ec, 
    -0x65d, 

    -0x1e7, 0xcc, 0x38b, 0x544, 0x6b6, 0x7d8, 0x7cc, 0x778, 0x68d, 0x4d6, 0x2ce, 0x78, -0x216, -0x42e, -0x5c7, -0x6d4, 
    -0x703, -0x6a5, -0x5a0, -0x409, -0x1f5, 0x8e, 0x355, 0x52a, 0x68e, 0x7b8, 0x7b6, 0x75b, 0x66d, 0x4e3, 0x2cd, 0x6c, 
    -0x22f, -0x488, -0x644, -0x789, -0x7da, -0x7c2, -0x6fb, -0x5a2, -0x3b1, -0x153, 0x15e, 0x37e, 0x4a3, 0x5f0, 0x63d, 0x5ed, 
    0x537, 0x3d1, 0x1fc, -0x34, -0x29a, -0x508, -0x6c4, -0x812, -0x8b5, -0x889, -0x7ca, -0x683, -0x4a0, -0x23d, 0x8f, 0x33a, 
    0x4de, 0x66f, 0x728, 0x711, 0x6d4, 0x5b8, 0x42a, 0x223, -0x1b, -0x28b, -0x448, -0x58a, -0x657, -0x61d, -0x551, -0x40d, 
    -0x244, 0x17, 0x2c6, 0x5d4, 0x7a6, 0x8ff, 0xa46, 0xa63, 0xa1f, 0x959, 0x7e6, 0x609, 0x41f, 0x28d, 0x18b, 0x10e, 
    0x157, 0x1f4, 0x2dc, 0x442, 0x440, 0x425, 0x3c2, 0x241, 0xd8, -0x105, -0x25a, -0x338, -0x395, -0x304, -0x238, -0xb4, 
    0xa9, 0xa6, 0xe3, 0x57, -0x11d, -0x2a8, -0x4e7, -0x773, -0xa48, -0xcc4, -0xea8, -0x1024, -0x10ce, -0x10d0, -0x1043, -0xef3, 
    -0xd1b, -0x

    0x50, -0xad, -0x147, -0x188, -0x180, -0xf1, -0x12, 0x106, 0x1b9, 0x1dd, 0x1d5, 0x14e, 0x5b, -0xa4, -0x13f, -0x180, 
    -0x177, -0xe3, 0x2, 0x121, 0x1dc, 0x201, 0x1fe, 0x177, 0x7e, -0x88, -0x12d, -0x177, -0x17a, -0xf5, -0x17, 0x100, 
    0x1bc, 0x1e7, 0x1e8, 0x171, 0x84, -0x73, -0x10e, -0x14f, -0x14b, -0xc4, 0x17, 0x12c, 0x1e3, 0x207, 0x207, 0x186, 
    0x9a, -0x64, -0x107, -0x14d, -0x152, -0xd0, 0x6, 0x117, 0x1c4, 0x1e2, 0x1d5, 0x14b, 0x51, -0xb4, -0x15b, -0x1a1, 
    -0x1a1, -0x11b, -0x46, 0xc5, 0x16e, 0x17d, 0x163, 0xd4, -0x26, -0x126, -0x1c3, -0x207, -0x20d, -0x196, -0xd2, 0x2f, 
    0xd9, 0xf3, 0xec, 0x69, -0x9b, -0x1cc, -0x303, -0x41c, -0x504, -0x5a1, -0x5df, -0x5c2, -0x54c, -0x484, -0x377, -0x233, 
    -0xcb, 0xa9, 0x20d, 0x34f, 0x457, 0x503, 0x539, 0x502, 0x464, 0x372, 0x24f, 0x13b, 0x83, 0x23, 0x12, 0x75, 
    0x129, 0x217, 0x2b0, 0x2be, 0x2a6, 0x219, 0x11b, 0xa, -0xb1, -0x112, -0x12d, -0xc3, 0x1, 0x104, 0x1b2, 0x1cb, 
    0x1ba, 0x131, 0x35, -0xd8, -0x18a, -0x1db, -0x1e9,

    0x206, 0x203, 0x18c, 0xa1, -0x63, -0x109, -0x147, -0x149, -0xc8, 0x9, 0x118, 0x1dc, 0x206, 0x20a, 0x1a6, 0xcc, 
    -0x29, -0xcc, -0x113, -0x11c, -0xa6, 0x3a, 0x16a, 0x2b6, 0x3bb, 0x472, 0x4f5, 0x50d, 0x4aa, 0x3f0, 0x2ee, 0x1b2, 
    0x58, -0x10a, -0x262, -0x39d, -0x4a5, -0x564, -0x5c8, -0x5c6, -0x55e, -0x498, -0x384, -0x249, -0x122, -0x5a, 0xe, 0x1b, 
    -0x52, -0x113, -0x1f1, -0x282, -0x2b0, -0x2a7, -0x224, -0x14c, -0x3b, 0x9b, 0xdc, 0x100, 0xb9, -0x8, -0xef, -0x192, 
    -0x1db, -0x1e4, -0x16d, -0x89, 0xb9, 0x234, 0x3a6, 0x4e5, 0x5ee, 0x6b3, 0x715, 0x713, 0x6b8, 0x60f, 0x51e, 0x3eb, 
    0x290, 0x126, -0x35, -0x16b, -0x255, -0x2e5, -0x311, -0x2d7, -0x235, -0x14e, -0x4c, 0x5f, 0x7f, 0x6e, -0x12, -0xfa, 
    -0x200, -0x2ba, -0x318, -0x340, -0x2ef, -0x246, -0x158, -0xa5, -0x83, -0x88, -0xf9, -0x1ea, -0x305, -0x3fa, -0x4a1, -0x50d, 
    -0x512, -0x4aa, -0x3e3, -0x2c3, -0x16b, 0x6, 0x16f, 0x2b6, 0x3d2, 0x4b1, 0x534, 0x550, 0x508, 0x464, 0x370, 0x245, 
    0x105, -0x37, -0x157, -0x24

    0x1e3, 0x84, -0xde, -0x228, -0x347, -0x427, -0x4bb, -0x4fe, -0x4eb, -0x481, -0x3c5, -0x2c3, -0x18e, -0x4b, 0xe4, 0x1e5, 
    0x2a3, 0x2fa, 0x2db, 0x251, 0x177, 0x6d, -0x68, -0xce, -0xf7, -0xb5, -0xa, 0xe3, 0x1cb, 0x20f, 0x20e, 0x1c1, 
    0xe5, -0x34, -0x121, -0x19f, -0x1e1, -0x1a7, -0xfc, -0x7, 0xe1, 0x123, 0x120, 0xd0, -0x9, -0x111, -0x1e0, -0x239, 
    -0x258, -0x208, -0x151, -0x61, 0x80, 0xb9, 0xba, 0x71, -0x58, -0x156, -0x222, -0x282, -0x2a6, -0x257, -0x193, -0x78, 
    0xe6, 0x240, 0x361, 0x457, 0x514, 0x56e, 0x560, 0x4f2, 0x434, 0x32b, 0x1ec, 0x92, -0xc6, -0x204, -0x30c, -0x3d1, 
    -0x43b, -0x447, -0x3f6, -0x345, -0x243, -0x103, 0x62, 0x1c3, 0x304, 0x411, 0x4d5, 0x534, 0x528, 0x4bb, 0x3f9, 0x2f3, 
    0x1bc, 0x6f, -0xe0, -0x215, -0x31b, -0x3dc, -0x444, -0x451, -0x3ff, -0x349, -0x24b, -0x121, -0x29, 0x4b, 0x86, 0x65, 
    -0x37, -0x107, -0x1b2, -0x1f8, -0x206, -0x1ab, -0xe1, 0x2d, 0x12e, 0x18b, 0x1a0, 0x166, 0x9b, -0x65, -0x137, -0x193, 
    -0x1ae, -0x155, -0x91, 0x71, 0x1

    0x3bb, 0x2b1, 0x17f, 0x45, -0xe3, -0x1e5, -0x2ae, -0x32a, -0x356, -0x322, -0x28f, -0x1a8, -0x78, 0xe1, 0x249, 0x39a, 
    0x4c0, 0x5b3, 0x662, 0x6b1, 0x6a0, 0x631, 0x570, 0x465, 0x331, 0x1f5, 0xd2, -0x14, -0xa2, -0xd0, -0x98, -0x1, 
    0xd5, 0x19e, 0x1d2, 0x1b6, 0x15f, 0x8a, -0x85, -0x170, -0x1eb, -0x22e, -0x214, -0x185, -0xb4, 0x30, 0x78, 0x70, 
    0x29, -0xb7, -0x1e7, -0x32e, -0x454, -0x54d, -0x604, -0x661, -0x663, -0x608, -0x558, -0x45b, -0x321, -0x1c1, -0x5a, 0xf8, 
    0x223, 0x313, 0x3aa, 0x3d7, 0x399, 0x300, 0x21b, 0xfd, -0x38, -0x16c, -0x28a, -0x383, -0x441, -0x4b2, -0x4d4, -0x49b, 
    -0x407, -0x320, -0x1f5, -0x9d, 0xc3, 0x20a, 0x328, 0x40b, 0x498, 0x4c2, 0x48e, 0x401, 0x328, 0x211, 0xd9, -0x69, 
    -0x19d, -0x2b0, -0x381, -0x401, -0x424, -0x3e4, -0x342, -0x24b, -0x113, 0x52, 0x1c1, 0x31c, 0x455, 0x556, 0x600, 0x63a, 
    0x607, 0x56d, 0x47c, 0x353, 0x21f, 0x126, 0x92, 0x52, 0x81, 0x114, 0x1e9, 0x2c5, 0x307, 0x2ef, 0x2a2, 0x1d3, 
    0xbf, -0x34, -0xba, -0x108, -0xf3, 

    0x26a, 0x126, -0x31, -0x183, -0x2b7, -0x3b7, -0x472, -0x4de, -0x4ec, -0x49d, -0x3f3, -0x2fa, -0x1c3, -0x6c, 0xe8, 0x220, 
    0x32b, 0x3f9, 0x46d, 0x478, 0x422, 0x36e, 0x26c, 0x132, -0x23, -0x172, -0x2a4, -0x3a0, -0x452, -0x4ae, -0x4ac, -0x44a, 
    -0x393, -0x28e, -0x150, 0x10, 0x16b, 0x2a8, 0x3b6, 0x480, 0x4ec, 0x4ea, 0x485, 0x3c9, 0x2c4, 0x18d, 0x45, -0xff, 
    -0x22f, -0x336, -0x3fc, -0x470, -0x489, -0x443, -0x3a0, -0x2ae, -0x182, -0x46, 0xc6, 0x174, 0x1c9, 0x1b3, 0x125, 0x4e, 
    -0x93, -0x10f, -0x13c, -0x11c, -0x79, 0x77, 0x194, 0x23d, 0x265, 0x262, 0x1d0, 0xd4, -0x2d, -0xc9, -0x116, -0x112, 
    -0x8a, 0x49, 0x148, 0x1c3, 0x1bd, 0x18c, 0xd2, -0x40, -0x149, -0x1e6, -0x231, -0x232, -0x1b9, -0xf8, -0xd, 0x64, 
    0x5b, 0x32, -0x7e, -0x18e, -0x2ad, -0x387, -0x419, -0x46b, -0x44b, -0x3c3, -0x2de, -0x1ae, -0x53, 0x10e, 0x25b, 0x382, 
    0x471, 0x515, 0x555, 0x52b, 0x4a6, 0x3c8, 0x2a5, 0x15d, 0xe, -0x12e, -0x245, -0x31e, -0x3aa, -0x3de, -0x3b4, -0x329, 
    -0x247, -0x116, 0x46

    -0xbd8, -0xb93, -0xacc, -0xbd5, -0xb72, -0xacd, -0x887, -0x52a, -0x4e4, -0x380, -0x160, 0xbb, 0x277, 0x3fb, 0x5d9, 0x8ea, 
    0x948, 0xa8a, 0xda4, 0xda3, 0xe79, 0xffb, 0x116a, 0x1131, 0x12a4, 0x13d5, 0x142b, 0x14ed, 0x148d, 0x1599, 0x155d, 0x14df, 
    0x1347, 0x132f, 0x130b, 0x1167, 0x11f8, 0x1016, 0xf2b, 0xfdc, 0xf66, 0xfda, 0x105c, 0xfff, 0xfac, 0xfcd, 0x10a2, 0x11ba, 
    0x11fe, 0x1167, 0xfb0, 0x1016, 0xfaf, 0xded, 0xd4a, 0xa92, 0x7f6, 0x554, 0x37c, 0xe6, -0x2f0, -0x4f9, -0x760, -0xa28, 
    -0xb20, -0xc32, -0xdf5, -0xeb2, -0x1056, -0x1142, -0x13e2, -0x146e, -0x14b7, -0x15fd, -0x140b, -0x1378, -0x139d, -0x13c4, -0x12a7, -0x12f0, 
    -0x121b, -0x122d, -0x1474, -0x1369, -0x1232, -0x119a, -0x11f7, -0x1187, -0xf5a, -0xf9d, -0xfbb, -0xe9e, -0xe44, -0xcd1, -0xa9e, -0x954, 
    -0x6da, -0x676, -0x62c, -0x522, -0x33f, -0x3e, -0x121, 0xaa, 0x32e, 0x4e1, 0x656, 0x853, 0x91a, 0xa22, 0xb22, 0xb60, 
    0xbd2, 0xce1, 0xe58, 0xca0, 0xc78, 0xbb4, 0xc32, 0xa62, 0x8db, 0x93d, 0x857, 0x88d, 0

The data above represents the PLL output from the same 10ms of data we have been testing with during this development process.  The values represent the input, sample, lock.

## Vivado HLS

WWe are going to make the biggest additions to the code since we started.  We will continue to use core pieces we created earlier, but we now add the digital PLL.  This requires two additional components: an IIR filter and hysteresis.  For these components, which in Python are implemented using floating point types, we are going to switch to 18-bit fixed point.  Why 18 bits?  Because that is the limit to the DSP48 blocks on the Zynq.  And initial results show that it worked.

If you would like to learn more about the capabilities of the DSP blocks in Zynq, the DSP48 User Guide from Xilinx is very detailed: https://www.xilinx.com/support/documentation/user_guides/ug479_7Series_DSP48E1.pdf

 1. Start Vivado HLS.
    ```bash
    vivado_hls
    ```
 1. Create a new project under the project_04 directory call HLS.
 1. Create a top-level function called demodulate4.
 1. Create 5 new files:
    * [demodulate.hpp](HLS/demodulate.hpp)
    * [demodulate.cpp](HLS/demodulate.cpp)
    * [hysteresis.hpp](HLS/hysteresis.hpp)
    * [iir_filter.hpp](HLS/iir_filter.hpp)
    * [digital_pll.hpp](HLS/digital_pll.hpp)    
 1. Create a new test bench:
    * [demodulate_test.cpp](HLS/demodulate_test.cpp)
 
The important part of this module is the addition of the three new header files which implement the digital PLL.  These work exactly the same as the digital PLL from the Python implementation.  The bulk of the code was copied from the [Mobilinkd TNC3 firmware](https://github.com/mobilinkd/tnc3-firmware) and modifies slightly for fixed-point math.

-----

This is the header:

```c++
#include <ap_axi_sdata.h>
#include <hls_stream.h>
#include <stdint.h>

#define BPF_COEFF_LEN 141

typedef ap_axis<16,1,1,1> idata_type;
typedef ap_axis<1,1,1,1> odata_type;

void demodulate5(idata_type input, odata_type& output);

```

The only change we needed to make here is to change the top-level function name.

And this is the source:

```c++
#include "demodulate.hpp"
#include "digital_pll.hpp"

#include "ap_shift_reg.h"

const ap_int<13> bpf_coeffs[] =
{    0,     0,     0,     0,     0,     0,     1,     3,     5,     8,     8,     5,
    -2,   -13,   -27,   -40,   -46,   -44,   -32,   -12,    11,    32,    44,    44,
    32,    14,     0,    -2,    13,    49,    97,   143,   170,   160,   104,     6,
  -118,  -244,  -340,  -381,  -352,  -258,  -120,    24,   138,   192,   173,    97,
     0,   -67,   -56,    62,   287,   575,   850,  1021,  1001,   737,   228,  -462,
 -1216, -1879, -2293, -2336, -1956, -1182,  -133,  1008,  2030,  2736,  2988,  2736,
  2030,  1008,  -133, -1182, -1956, -2336, -2293, -1879, -1216,  -462,   228,   737,
  1001,  1021,   850,   575,   287,    62,   -56,   -67,     0,    97,   173,   192,
   138,    24,  -120,  -258,  -352,  -381,  -340,  -244,  -118,     6,   104,   160,
   170,   143,    97,    49,    13,    -2,     0,    14,    32,    44,    44,    32,
    11,   -12,   -32,   -44,   -46,   -40,   -27,   -13,    -2,     5,     8,     8,
     5,     3,     1,     0,     0,     0,     0,     0,     0,
};

const ap_int<12> lpf_coeffs[] =
{
    0,    0,    0,    1,    3,    5,    8,   11,   14,   17,   20,   21,   20,   17,
   11,    2,   -9,  -25,  -44,  -66,  -91, -116, -142, -167, -188, -205, -215, -217,
 -209, -190, -156, -109,  -47,   30,  123,  230,  350,  481,  622,  769,  919, 1070,
 1217, 1358, 1488, 1605, 1704, 1785, 1844, 1880, 1893, 1880, 1844, 1785, 1704, 1605,
 1488, 1358, 1217, 1070,  919,  769,  622,  481,  350,  230,  123,   30,  -47, -109,
 -156, -190, -209, -217, -215, -205, -188, -167, -142, -116,  -91,  -66,  -44,  -25,
   -9,    2,   11,   17,   20,   21,   20,   17,   14,   11,    8,    5,    3,    1,
	0,    0,    0,
};

template <typename InOut, typename Filter, size_t N>
InOut fir_filter(InOut x, Filter (&coeff)[N])
{
    static InOut shift_reg[N];

    int32_t accum = 0;
    filter_loop: for (size_t i = N-1 ; i != 0; i--)
    {
#pragma HLS unroll factor=20
        shift_reg[i] = shift_reg[i-1];
        accum += shift_reg[i] * coeff[i];
    }

    shift_reg[0] = x;
    accum += shift_reg[0] * coeff[0];

    return static_cast<InOut>(accum >> 15);
}

ap_shift_reg<bool, 12> delay_line;
DigitalPLL<> dpll(26400, 1200);

void demodulate5(idata_type& input, odata_type& output)
{
#pragma HLS INTERFACE axis port=input
#pragma HLS INTERFACE axis port=output
#pragma HLS interface ap_ctrl_none port=return

	ap_int<16> bpfiltered, lpfiltered;
	ap_int<1> comp, delayed, comp2;
	ap_int<2> corr;

	bpfiltered = fir_filter(input.data, bpf_coeffs);
	comp = bpfiltered >= 0 ? 1 : 0;
	delayed = delay_line.shift(comp);
	corr = comp ^ delayed;
	corr <<= 1;
	corr -= 1;
	lpfiltered = fir_filter(corr, lpf_coeffs);
	comp2 = lpfiltered >= 0 ? 1 : 0;
	typename DigitalPLL<>::result_type result = dpll(comp2 != 0);

	ap_int<3> tmp = (std::get<0>(result) << 2) |
			(std::get<1>(result) << 1) | std::get<2>(result);
	output.data = tmp;
    output.dest = input.dest;
    output.id = input.id;
    output.keep = input.keep;
    output.last = input.last;
    output.strb = input.strb;
    output.user = input.user;
}
```


### C++11

Like before, we needed to add a configuration setting to control the timing contstraints.  In Vivado HLS, right click on the "solution1" window and select "Solution Settings...".  In the *Solution Settings* window, in the *General* tab, click the *Add* button.  Add a "config_core" setting for core "DSP48" with a latency of 3.  This is required to meet timing constraints with the new code.

We also use some new C++11 features -- specifically tuples.  For this we need to add compilation flags for use during simulation and synthesis.  Right click on the "HLS" project name in the Explorer window on the right side of the Vivado HLS UI and select "Project Settings...".  In the *Project Settings* window, select the *Similation* tab.  Then select the "demodulate_test.cpp" file.  Click the *Edit CFLAGS* button and add "-std=c++11" to the flags.  Go the to *Synthesis* tab, highlight the "demodulate.cpp" file and make the same change.

-----

Once the code and test bench are written, we need to run the C simulation, C synthesis, C/RTL co-simulation, then package the IP.  The two simulation steps run our test bench.  This verifies that the code will sythesize properly and that it functions properly.  For a software engineer, this is the same as compiling and running unit tests.

Once the IP is packaged, we are done in HLS.

## Vivado

We will now switch over to Vivado and create a block design.  These steps should start to feel very familiar to you by now.

 1. Start Vivado and create a new project.
 1. Give it a path -- in our case `afsk-demodulator-pynq/project_05` and the name `Vivado`.
 1. Select the `RTL Project` project type.
 1. In the "Default Part" screen, switch to the "Boards" tab. Select the your board from the list.
 1. Click "Finish".
 
With the new project open in Vivado, we need to create a block design.  We are going to follow the exact some procedure we did in the first three.

 1. On the right side, in the Flow Navigator, select *Create Block Diagram*.
 1. Use the default name, design_1.
 1. Go into Tools|Settings.
    1. In the settings dialog, choose IP|Repository.
    1. Select "+" to add a repository.
    1. Add Project_05/HLS as a repository.  You should see that it has 1 IP called `demodulate5` in there.
    1. When done, click "OK".
 1. In the Diagram view (main window) select "+" to add IP.
 1. Add the Zynq processing system and run block automation.
 1. When done, double-click the Zynq block and find the *High-performance AXI Slave Ports*.
 1. Click on the High-performance AXI Slave Ports.
 1. Enable the *S AXI HP0 interface*, then click OK.
 1. Add an AXI Stream Interconnect, AXI Direct Memory Access and the demodulator IP.
 1. Open the AXI Direct Memory Access, disable scatter/gather, and set the stream widths to 16 bits.
 1. Wire up the demodulator to the AXI Direct Memory Access and run connection automation.
    * A few additional modules are added: AXI SmartConnect, AXI Interconnect, and Processor System Reset
![BlockDesign](BlockDesign.png)
 1. Rename the demodulator block to "demodulate" and the DMA block to "dma".
 1. Combine the demodulate and dma blocks into a hierarchy called "demodulator".
 1. Generate the HDL wrapper by clicking on the design in the Sources box, right clicking, and selecting "Generate HDL Wrapper".
 1. Generate the bitstream. Again, this will take some time.
 1. Export the block design (File|Export|Export Block Design...)
 1. Collect the following files:
    - Vivado.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh
    - Vivado.runs/impl_1/design_1_wrapper.bit
    - design_1.tcl
    * rename these file to "project_03.{ext}" so that you have project_05.bit, project_05.tcl and project_05.hwh
 1. On the mounted Pynq filesystem, copy these files to `pynq/overlays/afsk_demodulator/`.
    ```bash
cp project_05.{tcl,bit,hwh} /var/run/media/${USER}/PYNQ/pynq/overlays/afsk_demodulator/
```
 1. You can now jump to the Jupyter notebook on the Pynq device.